### Version 1

In [6]:
import pandas as pd, numpy as np, os, sys 
from pprint import pprint 
import time, math 


sys.path.append('./')
sys.path.append('../')
sys.path.append('../../')


### Globals 

In [7]:
data_sources = {'customer_df_url' :'https://xavierpublic.blob.core.windows.net/public/olist/olist_customers_dataset.csv',
'order_reviews_url' :'https://xavierpublic.blob.core.windows.net/public/olist/olist_order_reviews_dataset.csv'
}

### Support 

In [8]:
from src.data_loaders.load_data_urls import load_url_raw_data

from src.azure_text_analytics.azure_transtation import get_text_translation_multiple_inputs,AzureKeyCredential, TextTranslationClient


### Load data

In [10]:
 
data =  load_url_raw_data( data_sources )
order_reviews_df = data['order_reviews_df']

display(order_reviews_df.sample(10))

order_reviews_df.shape 

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
69669,03540281387f7d1b1232860490635757,ff2af9c3633c70f37990bb59b0bed378,5,NaN,NaN,2018-04-06 00:00:00,2018-04-09 18:04:58
92464,ce8f854ec9e7502f8b9251cbd65a5250,cae717bb5d2448081dd91756a3842833,5,NaN,NaN,2018-07-21 00:00:00,2018-07-23 22:21:16
39521,949caba49070acd0a720073688b59fa3,9743ae8710712e56c645d9dbf4bb8fec,5,NaN,Tudo nas metragens e como vem na descrição mui...,2018-04-19 00:00:00,2018-04-20 01:56:45
40632,fd0e6c141e3e72b108a282747cfdb671,ed6104a0027e6a269ce0269f21c67f0c,4,NaN,Pensei q os produtos q pedi fossem com mais qu...,2017-08-05 00:00:00,2017-08-06 02:26:54
18056,6ac21af8cfe1d008e7e68427579862a6,390e5f12e01c2af04027580e4cdde515,5,NaN,Amei lindo recomendo sim,2018-04-24 00:00:00,2018-04-25 00:43:35
22620,2dfadb41598046188dc45c8fcecf6373,b39d5e5d31956b4598ed01674a0943e7,5,NaN,NaN,2018-08-17 00:00:00,2018-08-18 14:54:56
80199,6953c13980fc183745ea3fc0595c0d4e,869548743b57fc4906baa944c448a644,5,NaN,"ótimo produto, entrega rápida.",2017-10-12 00:00:00,2017-10-13 17:17:38
25257,8402f720969123b5c03bf6a0fc7a4981,2c440021d0192a2432f4662fc27225d4,4,40,"Muito bom produto, mas tecido inferior ao que ...",2018-06-20 00:00:00,2018-06-24 19:38:19
79100,dec5976bdb0c01b5ee70db58841558b4,94a280e9178a824b49777271fdf8e5cc,5,NaN,NaN,2018-06-27 00:00:00,2018-06-28 10:21:21
19601,2b0c7e87f8508baf2554b303422281cf,4b1130720b31500948a942732f0befc1,5,NaN,Recomendo,2017-10-25 00:00:00,2017-10-25 15:04:10


(99224, 7)

### Translate comments and title to English and add to the dataset 


In [48]:
reviews_df = order_reviews_df[ order_reviews_df['review_comment_message'].notnull() ].copy()
reviews_df.sample(5)

reviews_df

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01
...,...,...,...,...,...,...,...
99205,98fffa80dc9acbde7388bef1600f3b15,d398e9c82363c12527f71801bf0e6100,4,NaN,para este produto recebi de acordo com a compr...,2017-11-29 00:00:00,2017-11-30 15:52:51
99208,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,NaN,Entregou dentro do prazo. O produto chegou em ...,2018-02-07 00:00:00,2018-02-19 19:47:23
99215,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,NaN,"O produto não foi enviado com NF, não existe v...",2018-05-19 00:00:00,2018-05-20 21:51:06
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43


In [50]:
import pickle 
with open('data.pkl', 'rb') as file:
    # Load the pickled data
    recovered_data = pickle.load(file)

all = [ ]
for n in range( len(recovered_data)): all.extend( [item for item in  recovered_data[n] ] )

reviews_df['english_review'] = all 
reviews_df.to_csv('english_reviews.csv')

40977

In [ ]:
def normalize_text( s: str )->str:
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.replace("\r", "")
    s = '- ' + s.strip() + " -"
    
    return s

reviews_text = reviews_df['review_comment_message'].apply( lambda t: normalize_text(t) ).values.tolist()
reviews_text[0:5]

['- Recebi bem antes do prazo estipulado. -',
 '- Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa -',
 '- aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome..atualizar com a marca correta uma vez que é o mesmo aparelho -',
 '- Mas um pouco ,travando..pelo valor ta Boa. -',
 '- Vendedor confiável, produto ok e entrega antes do prazo. -']

In [7]:
from collections import defaultdict
from azure.ai.translation.text import TextTranslationClient#, TranslatorCredential
from azure.ai.translation.text.models import InputTextItem
from azure.core.exceptions import HttpResponseError
from azure.core.credentials import AzureKeyCredential

def get_block_translation_multiple_inputs(text_translator, input_text_elements:list[str], 
                                         to_language: list[str] =['en'] ):
    # [START get_text_translation_multiple_inputs]

    results = []
    if isinstance( to_language,str): to_language = [ to_language ]
    try:
        translations = text_translator.translate(body=input_text_elements, to_language=to_language)

        #print('done...')
        for n,translation in enumerate(translations):
            #print( translation )

            language_score = translation.detected_language.score 
            language_translation = translation.translations[0].text if translation.translations else None

            results.append( { input_text_elements[n] : { 'translation': language_translation, 'laguage_score': language_score}} ) 
            
            #print(
            #    f"Detected languages of the input text: {translation.detected_language.language if translation.detected_language else None} with score: {translation.detected_language.score if translation.detected_language else None}."
            #)
            #print(
            #    f"Text was translated to: '{translation.translations[0].to if translation.translations else None}' and the result is: '{translation.translations[0].text if translation.translations else None}'."
            #)

    except HttpResponseError as exception:
        if exception.error is not None:
            print(f"Error Code: {exception.error.code}")
            print(f"Message: {exception.error.message}")

        return None 
    
    except Exception as e: 
         print( str(e) )
         results.append( { 'input_text_elements[n]' : { 'translation': 'language translation failed', 'laguage_score': 0}} ) 
         return results   

    # [END get_text_translation_multiple_inputs]

    return results 
  

def split_text_array( text_array, chunk_size = 30000):
    sum = 0
    blocks = defaultdict( list )
    #chunk_size = chunk_size - 1 

    for n,t in enumerate(reviews_text):
        sum = sum + len( t ) #+ len( reviews_text[n+1] ) if n + 1 <= len(reviews_text) - 1  else 0 

        block_id =  math.floor (sum / chunk_size) # -1

        #print ( sum, block_id )
        blocks[ block_id ].append( t )
    
 


    x = [ list(blocks[key]) for key in blocks.keys() ] 
    return x 

    #return blocks


def get_text_translation_multiple_inputs(text_translator, input_text_elements:list[str], 
                                         to_language: list[str] =['en'] ):

    '''Notes: 
       Each translate request is limited to 50,000 characters, across all the target languages. 
       For example, sending a translate request of 3,000 characters to translate to three 
       different languages results in a request size of 3,000 × 3 = 9,000 characters and meets 
       the request limit
     '''
    
    # divide reviews_text into a number of arrays such that the sum of each is less that 50000
    blocks = split_text_array( input_text_elements )
    
    results = [] 
    for block_number, block in blocks.items():
        #print('\rblock ', block_number )
        partial_results = get_block_translation_multiple_inputs(text_translator, block,to_language )

        if not partial_results is None:
            results = results + partial_results 
            #print('block ', block_number, ' done ' )
    return results 




In [10]:
import pickle
 
for n, txt in enumerate(reviews_text[0:5]):
    print( n, txt )

# returns a list of lists. Each element, say blocks[0] is a list of strings.
# all the characters in those string add up to less than 30000, which is approx 
# the number of chars/minute that we can process within the limits of the service 
blocks = split_text_array( reviews_text, 30000 )

total_comments = sum ( len(x) for x in blocks )
print( total_comments, len(reviews_text), len(blocks) )
for k in range(5):print('chrs total in block  ', k,  sum( [ len(i) for i in blocks[k]] ))
print()

# so the idea now is to process one block per minute. Process -> store -> sleep 
results = [] 
key      = os.environ["TRANSLATOR_KEY"]
endpoint = os.environ["TRANSLATOR_ENPOINT"]
location = os.environ["TRANSLATOR_LOCATION"] 

text_translator = TextTranslationClient(credential=AzureKeyCredential( key ), region = location )

results = [] 
tot_chars = 0 
for n,block in enumerate(blocks): 
    chars_in_block = sum( [len(i) for i in block]) 
    tot_chars = tot_chars + chars_in_block
    print('processing block ', n,' chars in block ', chars_in_block, ' total ', tot_chars)
    r = get_block_translation_multiple_inputs(text_translator, block )
    results.append( r )
    print('...sleeping...', len(r), len(results))

    # Save object to a file
    with open("data.pkl", "wb") as file:
        pickle.dump(results, file)

    time.sleep(61)




0 - Recebi bem antes do prazo estipulado. -
1 - Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa -
2 - aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome..atualizar com a marca correta uma vez que é o mesmo aparelho -
3 - Mas um pouco ,travando..pelo valor ta Boa. -
4 - Vendedor confiável, produto ok e entrega antes do prazo. -
40977 40977
chrs total in block   0 29991
chrs total in block   1 30007
chrs total in block   2 29967
chrs total in block   3 29965
chrs total in block   4 30043

processing block  0  chars in block  29991  total  29991
...sleeping... 423 1
processing block  1  chars in block  30007  total  59998
...sleeping... 420 2
processing block  2  chars in block  29967  total  89965
...sleeping... 418 3
processing block  3  chars in block  29965  total  119930
...sleeping... 405 4
processing block  4  chars in block  30043  total  149973
...sleeping... 430 5
pr

TypeError: list indices must be integers or slices, not list

In [30]:
sum( [len(i) for i in recovered_data] ) 

 



40977

In [22]:
# we have about 40977 comments, which matches the initial number of comments 
comments_per_result = [ len( result ) for result in results ]
sum( comments_per_result )


40977

In [28]:
results[1]

[{'- Produto lindo, ótimo -': {'translation': '- Beautiful product, great -',
   'laguage_score': 1.0}},
 {'- excelente -': {'translation': '-excellent-', 'laguage_score': 0.89}},
 {'- Esperava mais, as toalhas poderiam ser mais grossas. -': {'translation': '- I expected more, the towels could be thicker. -',
   'laguage_score': 1.0}},
 {'- Entrega uma dia antes do previsto -': {'translation': '- Delivery one day ahead of schedule -',
   'laguage_score': 1.0}},
 {'- Demora um pouco pra esquentar, mesmo na potência máxima 5. Mas atende bem. Nos rimeiros dias de uso tem um cheiro meio que borracha queimada, mas passa.. recomendo pela praticidade e custo/benefício. -': {'translation': '- It takes a while to heat up, even at maximum power 5. But it serves well. On the first days of use it has a smell like burnt rubber, but it passes.. I recommend it for its practicality and cost/benefit. -',
   'laguage_score': 1.0}},
 {'- Foi entregue bem embalado.Entrega rapido comforme prometidoComforme

In [30]:
[ v['translation'] for result in results  for d in result for k,v in d.items()  ] 

['- I received it well before the stipulated deadline. -',
 '- Congratulations lannister stores loved buying over the Internet safe and practical Congratulations to all happy Easter -',
 "- Efficient device. On the website the brand of the device is printed as 3desinfector and when it arrives it is with another name.. update with the correct brand since it's the same handset –",
 "- But a little, crashing.. for the value it's Good. -",
 '- Reliable seller, ok product and delivery before the deadline. -',
 '- I WOULD LIKE TO KNOW WHAT HAPPENED, I ALWAYS RECEIVED AND THIS PURCHASE HAS NOW DISAPPOINTED ME -',
 '-Lousy-',
 '- Store note 10 -',
 '- Thank you for your attention -',
 '- The purchase was made easily. The delivery was made well before the deadline given. The product has already started to be used and so far, without problems. -',
 '- Very nice and cheap watch. -',
 "- I didn't like it! I bought a cat in a hare -",
 '- I always buy online and delivery occurs before the agreed de

In [24]:
[ i['translation'] for i in results[ 12 ] ]
    

KeyError: 'translation'

In [13]:
len( blocks )

99